In [77]:
import pandas as pd
import numpy as np
import os
from aocd import get_data

import sys
import time

from copy import deepcopy


In [60]:
puzzle = get_data(day=19, year=2023)

file = "/Users/liesbeth_bogaert/Documents/AoC/2023/input/input_day19.txt"
lines = open(file, "w").write(puzzle)

#puzzle = puzzle.splitlines()
puzzle = puzzle.split('\n\n')
flows, ratings = puzzle
#print(flows)

In [61]:
parts = []
for r in ratings.splitlines():
    input = r[1:-1].split(',')
    part = []
    for p in input:
        part.append(int(p[2:]))
    parts.append(part)

#print(parts)

In [119]:
workflows = {}
for w in flows.splitlines():
    part, rules = w.split('{')
    workflows[part] = rules[:-1]

#print(workflows)

Part 1

In [63]:
def is_A(part, work):
    w = workflows[work]
    #print(w)
    x, m, a, s = part
    for item in w.split(','):
        if item == 'A':
            return True
        if item == 'R':
            return False

        if ":" not in item:
            return is_A(part, item)
        
        cond = item.split(":")[0]
        #print(cond)
        if eval(cond):
            if item.split(":")[1] == 'R':
                return False
            if item.split(":")[1] == 'A':
                return True
            return is_A(part, item.split(":")[1])

        

In [64]:
total = 0

for part in parts:
    if is_A(part, 'in'):
        total += sum(part)

print(total)


421983


Part 2

In [120]:
workflows = {}
for line in flows.splitlines():
    l = line.split("{")
    name = l[0]
    parts = [part.split(":") for part in l[1][:-1].split(",")]
    parts = [
        [(part[0][0], part[0][1], int(part[0][2:])), part[1]]
        if len(part) == 2
        else part
        for part in parts
    ]
    workflows[name] = parts

#print(workflows)

In [115]:
def range_size(range):
    result = 1
    for val in range.values():
        result *= 1 + val[1] - val[0]
    return result


def solve(workflows, ranges, work):
    parts = workflows[work]
    val = 0

    for part in parts:
        if len(part) == 1:
            if part[0] == 'A':
                val += range_size(ranges)
            elif part[0] != 'R':
                val += solve(workflows, ranges, part[0])
        else:
            (xmas, cond, amount) = part[0]
            destination = part[1]
            range_xmas = ranges[xmas]

            if cond == '>':
                if range_xmas[1] > amount:
                    range_copy = deepcopy(ranges)
                    range_copy[xmas] = (max(range_xmas[0], amount + 1), range_xmas[1])

                    if destination == 'A':
                        val += range_size(range_copy)
                    elif destination != 'R':
                        val += solve(workflows, range_copy, destination)

                ranges[xmas] = (range_xmas[0], amount)
            else:
                if range_xmas[0] < amount:
                    range_copy = deepcopy(ranges)
                    range_copy[xmas] = (range_xmas[0], min(range_xmas[1], amount - 1))

                    if destination == 'A':
                        val += range_size(range_copy)
                    elif destination != 'R':
                        val += solve(workflows, range_copy, destination)

                ranges[xmas] = (amount, range_xmas[1])

    return val


In [116]:
ranges = {}
for val in "xmas":
    ranges[val] = (1, 4000)

total = solve(workflows, ranges, "in")

print(total)

129249871135292
